# **`HyperParameter Tuning `**

- **Hyperparameters** or **knobs** are parameters in ML algo that are set by the user before training and dictate complexity and variation of model 
- Therefore, Hyperparameter Tuning is a crucial process of finding the best **hyperparameters settings** for a ML model.
- It is done by trying **different** values for evaluation of model's performance

- In **sklearn** we have 2 main functions:
    - `GridSearchCV`  --> testing all the permutations of the value ranges set for the hyperparameters
    - `RandomizedSearchCV` --> is an efficient alternative to grid search as it randomly samples from the parameter distribution instead of exhaustively trying all the combinations 

> Earlier you manually did: for max_depth in range(1,21) now, GridSearchCV / RandomizedSearchCV do this automatically

- Both techniques also offer **Cross-Validation** (that's why CV in their name)
- **Cross-Validation**: 
    - It is an important concept in ML as it involves **partitioning** dataset into multiple subsets (eg: 4 folds) then training and evaluating the    model accuracy multiple times (4 times in our example) each time using a different fold
    - It helps provide a more reliable estimate of model's performance 




![cv](https://www.mathworks.com/discovery/cross-validation/_jcr_content/mainParsys/image.adapt.full.medium.jpg/1718274806179.jpg)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import make_scorer, accuracy_score


In [2]:
X, y = make_classification(n_samples=9000,   #Row count 
                           n_features=18,    #Column count
                           n_informative=4,  #4 useful features/columns - actually help predict the target 
                           n_redundant=12,   #12 redundant columns in informative features 
                           random_state=2
                          )

# total columns = 18, out of which '4' are useful & '12' are redundant so the remaining 2 columns are just Noise

## Performing **Cross-Validation Only**
- It is a technique used to access how well a predictive model generalizes to a dataset
- Use when you know which hyperparameters are useful to your model otherwise it is easier to use `GridSeachCV` as it does both hyperparameter tuning and cross-validation

In [3]:
#define the model
dtc_model = DecisionTreeClassifier(random_state=2)      #using default hyperparameter

In [4]:
scores = cross_val_score(dtc_model, X, y, cv=5)         # cv=5 --> 5-fold Cross-validation 
print("Cross-validation scores: ", scores)
print("Mean Cross-validation scores: ", scores.mean())      #to get average performace score of all the 5 folds 

Cross-validation scores:  [0.87222222 0.865      0.87944444 0.85888889 0.87166667]
Mean Cross-validation scores:  0.8694444444444445


- What's Happening Above:
    - Your data is split into 5 equal parts (**folds**)
    - The model is trained 5 times
    - Each time, 4 parts are used to train, 1 part is used to test

Each run gives you a score (usually accuracy by default) therefore we get 5 numbers where **each number = model performance on one fold**

## Performing **`GridSearchCV`**  (Hyperparameter Tuning + Cross-Validation)

### **Line-by-line meaning**
- param_grid = {}
    - This is a **dictionary** where:

        - **key** = **hyperparameter name**
        - **value** = **list of all possible values to try**

1️⃣ **`criterion`**
'criterion': ['gini', 'entropy']

- What it controls:
How the tree decides the best split at each node

- GridSearch will try:
    - trees that use gini impurity
    - trees that use entropy (information gain)

📌 You’re asking: Which split rule works better for my data?

2️⃣ **`max_depth`**
'max_depth': [2, 4, 6, 7, 10]

- What it controls:
    - How complex the tree is

- GridSearch will try:
    - very shallow trees (2)
    - medium trees (4–7)
    - deep trees (10)

📌 You’re exploring underfitting → optimal → overfitting zones.

3️⃣ **`min_samples_split`**
'min_samples_split': [2, 5, 10, 15, 20]

- What it controls:
When a node is allowed to split

- Effect:
    - 2 → aggressive splitting (more complex)
    - 20 → conservative splitting (simpler, smoother tree)

📌 This is regularization to reduce overfitting.

4️⃣ **`min_samples_leaf`** 'min_samples_leaf':[1, 2, 3, 4, 5]

- What it controls:
Ensures the tree makes decisions using enough data, not random points.

- If **min_samples_leaf = 1**
→ the tree can make decisions using just 1 data point
→ model may memorize noise → overfitting

- If **min_samples_leaf = 5**
→ every final decision must be based on at least 5 data points
→ decisions become more reliable and stable


- What GridSearchCV does with this:
    - With this grid, sklearn will test:

        - 2 (criterion)
        - × 5 (max_depth)
        - × 5 (min_samples_leaf)
        - x 5 (min_sample_split)
= **250** different models (with all having different combinations of above hyperparameters)

And with cv = 5:
- 250 × 5 = **1250** trainings/permutations

**All of this is done automatically**.
**That's why, GridSearch is powerful and expensive**.

In [5]:
param_grid = {                                          # defining parameter grid 
                'criterion':['gini', 'entropy'],        # How the tree decides the best split: 'gini' or 'entropy'
                'max_depth':[2,4,6,7,10],               # How deep the tree is allowed to grow
                'min_samples_leaf':[1,2,3,4,5],         # Leaf = final decision, More samples in leaf = more trustworthy decision
                'min_samples_split':[2,5,10,15,20]      # Minimum number of samples required to split a node
}

> This defines what values you want to try for each hyperparameter

In [6]:
gs_dtc_model = GridSearchCV(DecisionTreeClassifier(),          # Base model for evaluation
                        param_grid = param_grid,         # The hyperparameter combinations to try
                        scoring = 'accuracy',            # Metric used to decide the “best” model (evaluation metric), used to judge which model is best
                        cv=5)                            # 5-fold cross-validation

In [7]:
gs_dtc_model.fit(X, y)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeClassifier()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'criterion': ['gini', 'entropy'], 'max_depth': [2, 4, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 5, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'accuracy'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and par

> Some users use X_train, y_train instead of X and y in case you have more data and would like to do additional steps for evaluation

Now to check which combination gave the highest average accuracy across cross-validation use:
- **`gs_dtc.best_params_`**
- **best_params_** returns the hyperparameter combination that achieved the highest cross-validated accuracy during grid search.

In [8]:
gs_dtc_model.best_params_

{'criterion': 'gini',
 'max_depth': 10,
 'min_samples_leaf': 3,
 'min_samples_split': 15}

> 📌 This combination gave the highest average accuracy across cross-validation.

Now to check the **best model** found during hyperparameter tuning:
- **`gs_dtc_model.best_estimator_`**
- It is the trained model you should actually use in production or evaluation

In [9]:
gs_dtc_model.best_estimator_ 

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",10
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",15
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",3
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the curren

**`gs_dtc_model.best_score_`**
- It is the mean cross-validation score
- Tells how good the best model was during GridSearch

In [10]:
gs_dtc_model.best_score_

0.889

- The hyperparameter above gives 88.8% for best parameters, but this score is the best among only the values we tested, not necessarily the global best.

- What's next:
    - We know now the best hyperparameters
    - What if there are better values closer to those numbers and they might give even better results but as we never gave them to GridSearch we never get to know what they could have done 
    - So, we can go back to our grid and modify it 
    - Example: max_depth of 10, performed best among [2,4,6,7,10], but there may be better values close to the current best that were not tested, like [9,11,12]
    - Refine the grid around the best value. Perform another GridSearch with a narrower range near the winning parameter.

    - Example:

       - Initial grid (**coarse search**):
         max_depth = [2, 4, 6, 7, 10]   out of which 10 came to be the best 

       - Refined grid (**fine search**):
         max_depth = [9, 10, 11, 12]

Predicting and Calculating Accuracy using the Best Model:
- To use the model with the best hyperparameters, use **best_estimator_**
- **`gs_dtc_model.best_estimator_.predict(X)`**
- Takes the best trained model found by GridSearch.
- Uses it to predict the output labels for the input data X.

In [11]:
y_pred = gs_dtc_model.best_estimator_.predict(X)

accuracy_score = accuracy_score(y, y_pred)
print("Accuracy score =", round(accuracy_score, 2))

Accuracy score = 0.94


What if we want to see all Trials/Permutations that happended ?
- The best way to see it, is to convert it into a **DataFrame** 
- **`gs_dtc_model.cv_results_`**
    - A dictionary containing detailed results of every hyperparameter combination tested during GridSearch

- **`pd.DataFrame(...)`**
    - Converts that dictionary into a tabular format (rows and columns).

In [12]:
pd.DataFrame(gs_dtc_model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.038923,0.000898,0.001557,0.000170,gini,2,1,2,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
1,0.038987,0.001391,0.001516,0.000094,gini,2,1,5,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
2,0.037510,0.000394,0.001486,0.000175,gini,2,1,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
3,0.037860,0.000713,0.001572,0.000142,gini,2,1,15,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
4,0.037185,0.000921,0.001297,0.000096,gini,2,1,20,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0.200426,0.003057,0.001408,0.000044,entropy,10,5,2,"{'criterion': 'entropy', 'max_depth': 10, 'min...",0.885000,0.881667,0.883889,0.873889,0.884444,0.881778,0.004104,33
246,0.199522,0.003075,0.001668,0.000083,entropy,10,5,5,"{'criterion': 'entropy', 'max_depth': 10, 'min...",0.887778,0.878333,0.881667,0.871111,0.881667,0.880111,0.005437,58
247,0.204270,0.002901,0.001559,0.000214,entropy,10,5,10,"{'criterion': 'entropy', 'max_depth': 10, 'min...",0.886111,0.880000,0.883333,0.873333,0.884444,0.881444,0.004522,38
248,0.205449,0.003100,0.001567,0.000119,entropy,10,5,15,"{'criterion': 'entropy', 'max_depth': 10, 'min...",0.885556,0.882222,0.881667,0.870556,0.883889,0.880778,0.005289,46


- What this line did:
 -   Converts the complete GridSearch results into a pandas DataFrame
 -   Makes it easy to view, analyze, and compare all tested models

- What the DataFrame contains:

    - Each row represents one model configuration tried in GridSearch.
    - Common columns include:
        - **params** → the hyperparameters used
        - **mean_test_score** → average CV score
        - **std_test_score** → variation across folds
        - **rank_test_score** → ranking of models
        - **mean_fit_time** → training time
        - **split0_test_score, split1_test_score, …** → fold-wise scores

In [13]:
pd.DataFrame(gs_dtc_model.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,param_min_samples_leaf,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
113,0.177238,0.035552,0.001833,0.000331,gini,10,3,15,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.893889,0.882222,0.895000,0.877778,0.896111,0.889000,0.007515,1
100,0.154475,0.001776,0.001567,0.000129,gini,10,1,2,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.890556,0.886667,0.893333,0.877222,0.896667,0.888889,0.006694,2
118,0.153649,0.001845,0.001565,0.000151,gini,10,4,15,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.900000,0.885000,0.891111,0.874444,0.891111,0.888333,0.008433,3
105,0.154822,0.002311,0.001619,0.000144,gini,10,2,2,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.892222,0.891111,0.886111,0.872778,0.898333,0.888111,0.008597,4
109,0.153159,0.001354,0.001656,0.000192,gini,10,2,20,"{'criterion': 'gini', 'max_depth': 10, 'min_sa...",0.892778,0.883889,0.890556,0.876111,0.896111,0.887889,0.007118,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,0.037670,0.002186,0.001273,0.000156,gini,2,2,2,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
4,0.037185,0.000921,0.001297,0.000096,gini,2,1,20,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
2,0.037510,0.000394,0.001486,0.000175,gini,2,1,10,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
14,0.035985,0.000650,0.001351,0.000202,gini,2,3,20,"{'criterion': 'gini', 'max_depth': 2, 'min_sam...",0.704444,0.697222,0.707222,0.683333,0.713333,0.701111,0.010286,226
